# Open Electricity Data Toolkit — Quick Start

This notebook demonstrates the core workflow: **collect** electricity market data,
**store** it locally as Parquet, and **query** it back as clean DataFrames.

I'll use AESO (Alberta) data as an example. The same pattern works for IESO (Ontario).

In [ ]:
from elec_data import Toolkit

tk = Toolkit(data_dir="./data")

## 1. Collect AESO price data

The `collect()` method fetches data from the upstream API, stores it in
year-partitioned Parquet files, and logs the collection. Large date ranges
are automatically chunked by month.

In [ ]:
tk.collect(["AESO"], ["prices"], "2024-06-01", "2024-07-01")

## 2. Query prices

`get_prices()` reads from the local Parquet store. If the requested range
isn't stored yet, it will auto-fetch from the API.

In [ ]:
prices = tk.get_prices(["AESO"], "2024-06-01", "2024-07-01")
prices.head(10)

In [ ]:
print(f"Rows:  {len(prices):,}")
print(f"Range: {prices['timestamp_utc'].min()} → {prices['timestamp_utc'].max()}")
print(f"Mean price: ${prices['price'].mean():.2f}/MWh")

## 3. Plot prices

Alberta's pool price is set hourly and can spike dramatically during
periods of high demand or tight supply. The chart below shows the
intra-month price pattern for June 2024.

In [ ]:
import plotly.express as px

fig = px.line(
    prices,
    x="timestamp_utc",
    y="price",
    title="AESO Pool Price — June 2024",
    labels={"timestamp_utc": "Date (UTC)", "price": "Price (CAD/MWh)"},
    template="plotly_white",
)
fig.update_traces(line_width=0.8)
fig.show()

## 4. Collect and view demand data

In [ ]:
tk.collect(["AESO"], ["demand"], "2024-06-01", "2024-07-01")
demand = tk.get_demand(["AESO"], "2024-06-01", "2024-07-01")

fig = px.line(
    demand,
    x="timestamp_utc",
    y="demand_mw",
    title="AESO System Demand — June 2024",
    labels={"timestamp_utc": "Date (UTC)", "demand_mw": "Demand (MW)"},
    template="plotly_white",
)
fig.update_traces(line_width=0.8)
fig.show()

## 5. Check data status

`status()` summarizes what's in the local store — which markets and data
types are available, and their date ranges.

In [ ]:
tk.status()

## Next steps

- **Add more markets:** `tk.collect(["IESO"], ["prices"], "2024-01-01", "2024-07-01")`
- **Generation data:** `tk.get_generation(["AESO"], "2024-06-01", "2024-07-01")`
- **Multi-market comparison** and **resampling** are on the roadmap (Phase 2).

All data is stored locally as Parquet in `./data/raw/`.